<a href="https://colab.research.google.com/github/shivaansh81001/cmput-466-mini-project/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datascience import *
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [15]:
RANDOM_SEED = 420

TRAIN_PERC=0.6
VAL_PERC=0.2
TEST_PERC=0.2

In [16]:
#test imports
arr=[0,1,2,3,4,5,6,7,8,9]
print("numpy working ",np.array(arr))
print("pandas working ",pd.array(arr))
Table().with_column("test",np.array(arr)).show(1)


numpy working  [0 1 2 3 4 5 6 7 8 9]
pandas working  <IntegerArray>
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Length: 10, dtype: Int64


test
0


In [17]:
data=Table().read_table('cumulative.csv')
##data.show(5)
data=data.drop(["koi_tce_delivname","koi_pdisposition","kepler_name","rowid","kepid","kepoi_name","koi_teq_err1","koi_teq_err2","koi_score"])

for col in data.labels:
  data=data.where(col,are.not_equal_to('nan'))
  data=data.where(col,are.not_equal_to(float(np.nan)))

#print(type(data.take(99).column('koi_tce_plnt_num')[0]),data.take(99).column('koi_tce_plnt_num')[0])

#data.show(4)

In [18]:
data=data.to_df().dropna()
data=Table().from_df(data)

#data.group('koi_disposition').show(5)
#data.show(10)
num_false=data.where('koi_disposition',are.equal_to('FALSE POSITIVE')).num_rows


false_data=data.where('koi_disposition',are.equal_to('FALSE POSITIVE')).sample(k=int(0.5*num_false))
##false_data.show(3)


confirmed_data=data.where('koi_disposition',are.equal_to('CONFIRMED'))
#confirmed_data.show(3)

data= false_data.append(confirmed_data).shuffle()
data=data.shuffle()


target = (data.column("koi_disposition")=="CONFIRMED").astype(int)
X = data.drop("koi_disposition").to_df()

In [19]:
def generate_report(t_true,t_pred):
  return {
        'Accuracy': accuracy_score(t_true, t_pred),
        'Precision': precision_score(t_true, t_pred),
        'Recall': recall_score(t_true, t_pred),
        'F1-Score': f1_score(t_true, t_pred)
    }

In [20]:
def get_accuracy(t, t_hat):
    """
    Calculate accuracy,
    """
    #print(t,t_hat)
    acc=np.mean(t==t_hat)*100
    #print("accuracy",acc)
    return acc

In [21]:
def baseline_model(X_test,t_test):
  k = len(np.unique(t_test))
  #print(k)
  np.random.seed(RANDOM_SEED)
  t_hat=np.random.choice(np.unique(t_test),size=len(X_test),replace=True)

  return t_hat


In [22]:
def sigmoid(z):
    return 1 / (1 + np.exp(-np.clip(z, -400, 400)))

def train_logistic_regression(X, t):
    """
    Given data, train your logistic classifier.
    Return weight and bias

    """

    m,n=X.shape
    w=np.zeros(n)
    b=0

    for i in range(1000):
        z=np.dot(X,w)+b
        t_hat=sigmoid(z)
        dw=(1/m)*np.dot(X.T,(t_hat-t))
        db=(1/m)*np.sum(t_hat-t)
        w-=0.1*dw
        b-=0.1*db
    #print(w,b)

    return w, b

def predict_logistic_regression(X, w, b):
    """
    Generate predictions by your logistic classifier.
    """
    temp=1/(1+np.exp(-(X@w+b)))
    t=(temp>=0.5).astype(int)
    return t

In [23]:
def train_SVM(X_train,t_train):
  svm =SVC(kernel='rbf',C=10,random_state= RANDOM_SEED,gamma=0.001,class_weight='balanced')
  svm.fit(X_train,t_train)
  return svm

def predict_SVM(model_fit,X_val):
  t_hat=model_fit.predict(X_val)
  return t_hat


In [24]:
def train_MLP(X_train,t_train):
  mlp=MLPClassifier(hidden_layer_sizes=(64,32),activation='relu', max_iter=300, random_state=42)
  mlp.fit(X_train,t_train)
  return mlp

def predict_MLP(mlp,X_val):
  t_hat=mlp.predict(X_val)
  return t_hat


In [25]:

total_rows=len(target)

np.random.seed(RANDOM_SEED)

X_train, X_temp, t_train, t_temp = train_test_split(X, target, test_size=(1 - TRAIN_PERC), random_state=RANDOM_SEED,stratify=target)
X_val, X_test, t_val, t_test = train_test_split(X_temp, t_temp, test_size=0.5, random_state=RANDOM_SEED, stratify=t_temp)

In [28]:
def main():

  ##################### BASELINE #####################
  t_hat_test_base=baseline_model(X_test,t_test)
  print("Accuracy of baseline on test set: ", get_accuracy(t_hat_test_base, t_test))
  print(pd.DataFrame({
      "Test Set": generate_report(t_test,t_hat_test_base)
                         }))
  ##################### BASELINE #####################



  ##################### LOGISTIC REGRESSION #####################
  w_log, b_log = train_logistic_regression(X_train, t_train)
  t_hat_val_log = predict_logistic_regression(X_val, w_log, b_log)
  print("Accuracy of logistic regression on validation set: ", get_accuracy(t_hat_val_log, t_val))
  t_hat_test_log = predict_logistic_regression(X_test, w_log, b_log)
  print("Accuracy of logistic regression on test set: ", get_accuracy(t_hat_test_log, t_test))
  print(pd.DataFrame({
      "Validation Set": generate_report(t_val, t_hat_val_log),
      "Test Set": generate_report(t_test,t_hat_test_log)
                         }))
  ##################### LOGISTIC REGRESSION #####################



  ##################### SVM #####################
  model_fit = train_SVM(X_train,t_train)
  #print("Best params", model_fit.best_params_)
  t_hat_svm_val =predict_SVM(model_fit,X_val)
  print("Accuracy of SVM on validation set: ", get_accuracy(t_hat_svm_val, t_val))
  t_hat_svm_test= predict_SVM(model_fit,X_test)
  print("Accuracy of SVM on test set: ", get_accuracy(t_hat_svm_test, t_test))
  print(pd.DataFrame({
      "Validation Set": generate_report(t_val, t_hat_svm_val),
      "Test Set": generate_report(t_test,t_hat_svm_test)
                         }))
  ##################### SVM #####################



  ##################### MLP #####################
  mlp= train_MLP(X_train,t_train)
  t_hat_mlp_val=predict_MLP(mlp,X_val)
  print("Accuracy of MLP on validation set: ", get_accuracy(t_hat_mlp_val, t_val))
  t_hat_mlp_test= predict_MLP(mlp,X_test)
  print("Accuracy of MLP on test set: ", get_accuracy(t_hat_mlp_test, t_test))
  print(pd.DataFrame({
      "Validation Set": generate_report(t_val, t_hat_mlp_val),
      "Test Set": generate_report(t_test,t_hat_mlp_test)
                         }))
  ##################### SVM #####################


main()

Accuracy of baseline on test set:  48.9337822671
           Test Set
Accuracy   0.489338
Precision  0.500000
Recall     0.474725
F1-Score   0.487035


/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)


Accuracy of logistic regression on validation set:  71.3483146067
Accuracy of logistic regression on test set:  71.3804713805
           Validation Set  Test Set
Accuracy         0.713483  0.713805
Precision        0.890196  0.862319
Recall           0.500000  0.523077
F1-Score         0.640339  0.651163
Accuracy of SVM on validation set:  64.4943820225
Accuracy of SVM on test set:  64.1975308642
           Validation Set  Test Set
Accuracy         0.644944  0.641975
Precision        0.589610  0.588083
Recall           1.000000  0.997802
F1-Score         0.741830  0.740016
Accuracy of MLP on validation set:  88.6516853933
Accuracy of MLP on test set:  90.7968574635
           Validation Set  Test Set
Accuracy         0.886517  0.907969
Precision        0.937965  0.947242
Recall           0.832599  0.868132
F1-Score         0.882147  0.905963
